<a href="https://colab.research.google.com/github/RocioLiu/Python_coding/blob/master/BERT_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Refer to  [進擊的 BERT：NLP 界的巨人之力與遷移學習](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html)

# BERT：理解上下文的語言代表模型
一個簡單的 convention，等等文中會穿插使用的：

* 代表
* representation
* repr.
* repr. 向量
指的都是一個可以用來代表某詞彙（在某個語境下）的多維連續向量（continuous vector）。

In [1]:
%%bash
pip install transformers tqdm boto3 requests regex -q

ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "bert-base-chinese" # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [4]:
clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.8.1+cu101


In [5]:
# 看看 tokenizer 裡頭的字典資訊
vocab = tokenizer.vocab
print(f"字典大小：{len(vocab)}")

字典大小：21128


如上所示，中文 BERT 的字典大小約有 2.1 萬個 tokens

In [6]:
import random
random_tokens = random.sample(list(vocab), 10)
random_tokens

['##腸', 'comhd', '1942', '哽', '##穩', '##耻', '##撐', '##5', '389', '##００']

In [7]:
random_ids = [vocab[t] for t in random_tokens]
random_ids

[18648, 12587, 9480, 1531, 18009, 18516, 16109, 8157, 12512, 9885]

In [8]:
print(f"{'token': <20}{'index':<15}")
print('-'*25)
for t, id in zip(random_tokens, random_ids):
  print(f"{t:<20}{id:<10}")

token               index          
-------------------------
##腸                 18648     
comhd               12587     
1942                9480      
哽                   1531      
##穩                 18009     
##耻                 18516     
##撐                 16109     
##5                 8157      
389                 12512     
##００                9885      


BERT 使用當初 [Google NMT](https://ai.googleblog.com/2016/09/a-neural-network-for-machine.html) 提出的 [WordPiece Tokenization](https://arxiv.org/abs/1609.08144) ，將本來的 words 拆成更小粒度的 wordpieces，有效處理不在字典裡頭的詞彙 。中文的話大致上就像是 character-level tokenization，而有 ## 前綴的 tokens 即為 wordpieces。

ㄅㄆㄇㄈ也有被收錄：

In [9]:
indices = list(range(647, 657))
some_pairs = [(t, idx) for t, idx in vocab.items() if idx in indices]
for pair in some_pairs:
  print(pair)

('ㄅ', 647)
('ㄆ', 648)
('ㄇ', 649)
('ㄉ', 650)
('ㄋ', 651)
('ㄌ', 652)
('ㄍ', 653)
('ㄎ', 654)
('ㄏ', 655)
('ㄒ', 656)


利用中文 BERT 的 tokenizer 將一個中文句子斷詞看看：

In [10]:
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens)
print(ids)

[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知', '道', '誰', '沒', '穿', '褲', '子', '。']
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761, 6887, 6306, 3760, 4959, 6194, 2094, 511]


除了一般的 wordpieces 以外，BERT 裡頭有 5 個特殊 tokens：
* [CLS]：在做分類任務時其最後一層的 repr. 會被視為整個輸入序列的 repr.
* [SEP]：有兩個句子的文本會被串接成一個輸入序列，並在兩句之間插入這個 token 以做區隔
* [UNK]：沒出現在 BERT 字典裡頭的字會被這個 token 取代
* [PAD]：zero padding 遮罩，將長度不一的輸入序列補齊方便做 batch 運算
* [MASK]：未知遮罩，僅在預訓練階段會用到

如上例所示，[CLS] 一般會被放在輸入序列的最前面，而 zero padding 在之前的 [Transformer 文章](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html#%E7%9B%B4%E8%A7%80%E7%90%86%E8%A7%A3%E9%81%AE%E7%BD%A9%E5%9C%A8%E6%B3%A8%E6%84%8F%E5%87%BD%E5%BC%8F%E4%B8%AD%E7%9A%84%E6%95%88%E6%9E%9C) 裡已經有非常詳細的介紹。[MASK] token 一般在 fine-tuning 或是 feature extraction 時不會用到，這邊只是為了展示預訓練階段的克漏字任務才使用的。

現在讓我們看看給定上面有 [MASK] 的句子，BERT 會填入什麼字：

In [11]:
"""
這段程式碼載入已經訓練好的 masked 語言模型並對有 [MASK] 的句子做預測
"""
from transformers import BertForMaskedLM
# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids]) # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor) # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [12]:
tokens_tensor

tensor([[ 101, 5023, 1168, 4060, 3717,  103,  749, 8024, 2218, 4761, 6887, 6306,
         3760, 4959, 6194, 2094,  511]])

In [13]:
# 使用 masked LM 估計 [MASK] 位置所代表的實際 token
maskedLM_model.eval()
with torch.no_grad():
  outputs = maskedLM_model(tokens_tensor, segments_tensors)
  predictions = outputs[0] # (1, seq_len, num_hidden_units)

In [14]:
outputs

MaskedLMOutput([('logits',
                 tensor([[[ -9.9042,  -9.5542,  -9.8265,  ...,  -6.8384,  -6.1913,  -6.8853],
                          [ -8.5631,  -7.9794,  -8.6885,  ...,  -5.8659,  -4.6942,  -5.3190],
                          [-16.6086, -16.1292, -16.4605,  ..., -12.4272, -10.7403, -15.4416],
                          ...,
                          [-12.6074, -13.6432, -13.6270,  ..., -11.8602,  -8.6505, -15.0264],
                          [-14.8105, -15.8511, -15.1675,  ..., -10.1823,  -1.8096, -11.0179],
                          [-12.4678, -12.5876, -12.1389,  ...,  -9.1602,  -3.3892,  -9.0933]]]))])

In [15]:
predictions

tensor([[[ -9.9042,  -9.5542,  -9.8265,  ...,  -6.8384,  -6.1913,  -6.8853],
         [ -8.5631,  -7.9794,  -8.6885,  ...,  -5.8659,  -4.6942,  -5.3190],
         [-16.6086, -16.1292, -16.4605,  ..., -12.4272, -10.7403, -15.4416],
         ...,
         [-12.6074, -13.6432, -13.6270,  ..., -11.8602,  -8.6505, -15.0264],
         [-14.8105, -15.8511, -15.1675,  ..., -10.1823,  -1.8096, -11.0179],
         [-12.4678, -12.5876, -12.1389,  ...,  -9.1602,  -3.3892,  -9.0933]]])

In [16]:
predictions.size()

torch.Size([1, 17, 21128])

In [17]:
del maskedLM_model

In [18]:
# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5  # [mask] 位於 index 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

In [19]:
predicted_tokens

['來', '濕', '過']

In [25]:
# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print(f"輸入 tokens: {tokens[:10]} ...")
print('-' * 80)
for i, (t, p) in enumerate(zip(predicted_tokens, probs)):
  tokens[masked_index] = t
  print(f"Top {i} ({int(p.item() * 100):<2}%): {tokens[:10]} ...")

輸入 tokens: ['[CLS]', '等', '到', '潮', '水', '過', '了', '，', '就', '知'] ...
--------------------------------------------------------------------------------
Top 0 (67%): ['[CLS]', '等', '到', '潮', '水', '來', '了', '，', '就', '知'] ...
Top 1 (25%): ['[CLS]', '等', '到', '潮', '水', '濕', '了', '，', '就', '知'] ...
Top 2 (2 %): ['[CLS]', '等', '到', '潮', '水', '過', '了', '，', '就', '知'] ...


Google 在訓練中文 BERT 鐵定沒看批踢踢，還無法預測出我們最想要的那個 `退` 字。而最接近的 `過` 的出現機率只有 2%，但我會說以語言代表模型以及自然語言理解的角度來看這結果已經不差了。BERT 透過關注 `潮` 與 `水` 這兩個字，從 2 萬多個 wordpieces 的可能性中選出 `來` 作為這個情境下 `[MASK]` token 的預測值 ，也還算說的過去。

![](https://leemeng.tw/images/bert/bert-attention.jpg)
以上是 [BertViz](https://github.com/jessevig/bertviz) 視覺化 BERT 注意力的結果，等等列出安裝步驟。值得一提的是，以上是第 8 層 Encoder block 中 [Multi-head attention](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html#Multi-head-attention%EF%BC%9A%E4%BD%A0%E7%9C%8B%E4%BD%A0%E7%9A%84%EF%BC%8C%E6%88%91%E7%9C%8B%E6%88%91%E7%9A%84) 裡頭某一個 head 的自注意力結果。並不是每個 head 都會關注在一樣的位置。透過 multi-head 自注意力機制，BERT 可以讓不同 heads 在不同的 representation subspaces 裡學會關注不同位置的不同 repr.。

學會填克漏字讓 BERT 更好地 model 每個詞彙在不同語境下該有的 repr.，而 NSP (Net Sentence Prediction) 任務則能幫助 BERT model 兩個句子之間的關係，這在[問答系統 QA](https://zh.wikipedia.org/wiki/%E5%95%8F%E7%AD%94%E7%B3%BB%E7%B5%B1)、[自然語言推論 NLI](http://nlpprogress.com/english/natural_language_inference.html) 或是後面我們會看到的[假新聞分類任務](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html#%E7%94%A8-BERT-fine-tune-%E4%B8%8B%E6%B8%B8%E4%BB%BB%E5%8B%99)都很有幫助。

這樣的 word repr. 就是近年十分盛行的 [contextual word representation](https://www.youtube.com/watch?v=S-CspeZ8FHc) 概念。跟以往沒有蘊含上下文資訊的 [Word2Vec、GloVe](https://www.youtube.com/watch?v=8rXD5-xhemo) 等無語境的詞嵌入向量有很大的差異。用稍微學術一點的說法就是：

```
Contextual word repr. 讓同 word type 的 word token 在不同語境下有不同的表示方式；
而傳統的詞向量無論上下文，都會讓同 type 的 word token 的 repr. 相同。
```

直覺上 contextual word representation 比較能反映人類語言的真實情況，畢竟同個詞彙的含義在不同情境下相異是再正常不過的事情。在不同語境下給同個詞彙相同的 word repr. 這件事情在近年的 NLP 領域裡頭顯得越來越不合理。

為了讓你加深印象，讓我再舉個具體的例子：
```
情境 1：
胖虎叫大雄去買漫畫，回來慢了就打他。

情境 2：
妹妹說胖虎是「胖子」，他聽了很不開心。
```
很明顯地，在這兩個情境裡頭「他」所代表的語義以及指稱的對象皆不同。如果仍使用沒蘊含上下文 / 語境資訊的詞向量，機器就會很難正確地「解讀」這兩個句子所蘊含的語義了。


現在讓我們跟隨 [這個 Colab 筆記本](https://colab.research.google.com/drive/1g2nhY9vZG-PLC3w3dcHGqwsHBAXnD9EY) 安裝 BERT 的視覺化工具 `BertViz`，看看 BERT 會怎麼處理這兩個情境：

In [26]:
# 安裝 BertViz
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 1383, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1383 (delta 113), reused 118 (delta 61), pack-reused 1203
Receiving objects: 100% (1383/1383), 213.84 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (876/876), done.


In [28]:
import sys
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']

In [35]:
# import packages
from transformers import BertTokenizer, BertModel
from bertviz import head_view

In [54]:
# 在 jupyter notebook 裡頭顯示 visualzation 的 helper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))
  
clear_output()

Setup 以後就能非常輕鬆地將 BERT 內部的注意力機制視覺化出來：

In [55]:
# 記得我們是使用中文 BERT
model_version = 'bert-base-chinese'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
# 情境 1 的句子
sentence_a = "胖虎叫大雄去買漫畫，"
sentence_b = "回來慢了就打他。"

In [62]:
# 得到 tokens 後丟入 BERT 取得 attention
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
print(f"Inputs: \n{inputs}")
token_type_ids = inputs['token_type_ids'] # This discern two sentences with 0 and 1
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids = token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

Inputs: 
{'input_ids': tensor([[ 101, 5523, 5988, 1373, 1920, 7413, 1343, 6525, 4035, 4529, 8024,  102,
         1726,  889, 2714,  749, 2218, 2802,  800,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


encode_plus() :   
https://huggingface.co/transformers/v2.11.0/main_classes/tokenizer.html

```
Returns a dictionary containing the encoded sequence or sequence pair and additional information: 
the mask for sequence classification and the overflowing elements if a max_length is specified.
```

Return a Dictionary of shape:
```
{
    input_ids: list[int],
    token_type_ids: list[int] if return_token_type_ids is True (default)
    attention_mask: list[int] if return_attention_mask is True (default)
    overflowing_tokens: list[int] if a ``max_length`` is specified and return_overflowing_tokens is True
    num_truncated_tokens: int if a ``max_length`` is specified and return_overflowing_tokens is True
    special_tokens_mask: list[int] if ``add_special_tokens`` if set to ``True``
    and return_special_tokens_mask is True
}
```

With the fields:

* `input_ids`: list of token ids to be fed to a model

* `token_type_ids`: list of token type ids to be fed to a model

* `attention_mask`: list of indices specifying which tokens should be attended to by the model

* `overflowing_tokens`: list of overflowing tokens if a max length is specified.

* `num_truncated_tokens`: number of overflowing tokens a max_length is specified

* `special_tokens_mask`: if adding special tokens, this is a list of [0, 1], with 0 specifying special added tokens and 1 specifying sequence tokens.

Since we are feeding in two sentences at a time, BERT (and likely other model variants), expect some form of masking, which allows the model to discern between the two sequences, see [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertTokenizer.create_token_type_ids_from_sequences).



In [64]:
# 交給 BertViz 視覺化
head_view(attention, tokens)

# 注意：執行這段程式碼以後只會顯示下圖左側的結果。
# 為了方便你比較，我把情境 2 的結果也同時附上

<IPython.core.display.Javascript object>

![](https://leemeng.tw/images/bert/bert-coreference.jpg)

這是 BERT 裡第 9 層 Encoder block 其中一個 head 的注意力結果。

圖中的線條代表該 head 在更新「他」（左側）的 repr. 時關注其他詞彙（右側）的注意力程度。越粗代表關注權重（attention weights）越高。很明顯地這個 head 具有一定的 [指代消解（Coreference Resolution）](https://www.youtube.com/watch?v=i19m4GzBhfc) 能力，能正確地關注「他」所指代的對象。

要處理指代消解需要對自然語言有不少理解，而 BERT 在沒有標注數據的情況下透過自注意力機制、深度雙向語言模型以及「閱讀」大量文本達到這樣的水準，是一件令人雀躍的事情。

當然 BERT 並不是第一個嘗試產生 contextual word repr. 的語言模型。在它之前最知名的例子有剛剛提到的 [ELMo](https://allennlp.org/elmo) 以及 [GPT](https://github.com/openai/gpt-2)：

![](https://leemeng.tw/images/bert/bert_elmo_gpt.jpg)
<center>ELMo、GPT 以及 BERT 都透過訓練語言模型來獲得 contextual word representation</center>

ELMo 利用獨立訓練的雙向兩層 LSTM 做語言模型並將中間得到的隱狀態向量串接當作每個詞彙的 contextual word repr.；GPT 則是使用 Transformer 的 Decoder 來訓練一個中規中矩，從左到右的單向語言模型。你可以參考我另一篇文章： [直觀理解 GPT-2 語言模型並生成金庸武俠小說](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html) 來深入了解 GPT 與 GPT-2。

BERT 跟它們的差異在於利用 MLM（即克漏字）的概念及 Transformer Encoder 的架構，擺脫以往語言模型只能從單個方向（由左到右或由右到左）估計下個詞彙出現機率的窘境，訓練出一個雙向的語言代表模型。這使得 BERT 輸出的每個 token 的 repr. `Tn` 都同時蘊含了前後文資訊，真正的雙向 representation。

跟以往模型相比，BERT 能更好地處理自然語言，在著名的問答任務 [SQuAD2.0](https://rajpurkar.github.io/SQuAD-explorer/) 也有卓越表現：

![](https://leemeng.tw/images/bert/squad2.jpg)
<center>SQuAD 2.0 之前排行榜的前 5 名有 4 個有使用 BERT</center>

假如你仍然似懂非懂，只需記得：

```
BERT 是一個強大的語言代表模型，給它一段文本序列，它能回傳一段相同長度且蘊含上下文資訊的 word repr. 序列，對下游的 NLP 任務很有幫助。
```

有了這樣的概念以後，我們接下來要做的事情很簡單，就是將自己感興趣的 NLP 任務的文本丟入 BERT ，為文本裡頭的每個 token 取得有語境的 word repr.，並以此 repr. 進一步 fine tune 當前任務，取得更好的結果。

---

# 用 BERT fine tune 下游任務
我們在給 [所有人的 NLP 入門指南](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html) 碰過的 [假新聞分類任務](https://www.kaggle.com/c/fake-news-pair-classification-challenge/rules) 將會是本文拿 BERT 來做 fine-tuning 的例子。選擇這個任務的最主要理由是因為中文數據容易理解，另外網路上針對兩個句子做分類的例子也較少。

![](https://leemeng.tw/images/nlp-kaggle-intro/view-data-on-kaggle.jpg)
<center>給定假新聞 title1，判斷另一新聞 title2 跟 title1 的關係（同意、反對或無關）</center>

fine tune BERT 來解決新的下游任務有 5 個簡單步驟：

1. 準備原始文本數據
2. 將原始文本轉換成 BERT 相容的輸入格式
3. 在 BERT 之上加入新 layer 成下游任務模型
4. 訓練該下游任務模型
5. 對新樣本做推論

步驟 1、4 及 5 都跟訓練一般模型所需的步驟無太大差異。跟 BERT 最相關的細節事實上是步驟 2 跟 3：

* 如何將原始數據轉換成 BERT 相容的輸入格式？
* 如何在 BERT 之上建立 layer(s) 以符合下游任務需求？

事不宜遲，讓我們馬上以假新聞分類任務為例回答這些問題。[在之前的文章已經說明過](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html)，這個任務的輸入是兩個句子，輸出是 3 個類別機率的多類別分類任務（multi-class classification task），跟 NLP 領域裡常見的 [自然語言推論（Natural Language Inference）](https://paperswithcode.com/task/natural-language-inference/latest) 具有相同性質。